In [1]:
import pandas as pd
import numpy as np
import time
import arrow

In [2]:
job_start_time = time.time()
df_train = pd.read_csv('../../dataset/meal_service/train.csv')
df_raining = pd.read_csv('../../dataset/meal_service/raining_stats.csv')
df_temperature = pd.read_csv('../../dataset/meal_service/temperature_stats.csv')
df_test = pd.read_csv('../../dataset/meal_service/test.csv')
df_result_date = pd.DataFrame(df_test['일자'])

In [3]:
df_train = pd.merge(left=df_train, right=df_raining, how='left', on='일자')
df_train = pd.merge(left=df_train, right=df_temperature, how='left', on='일자')
df_test = pd.merge(left=df_test, right=df_raining, how='left', on='일자')
df_test = pd.merge(left=df_test, right=df_temperature, how='left', on='일자')


In [4]:
def get_is_holiday_lists(df):
    # pre-processing    
    is_yesterday_holiday_list = []
    is_tomorrow_holiday_list = []

    for i in range (0, len(df)):
        if df.iloc[i]['요일'] == '월':
            is_yesterday_holiday_list.append(1)
            if i < len(df)-1:
                is_tomorrow_holiday_list.append(0) if df.iloc[i+1]['요일'] == '화' else is_tomorrow_holiday_list.append(1)
            else:
                is_tomorrow_holiday_list.append(0)
        elif df.iloc[i]['요일'] == '화':
            if i < len(df)-1:
                is_tomorrow_holiday_list.append(0) if df.iloc[i+1]['요일'] == '수' else is_tomorrow_holiday_list.append(1)
            else:
                is_tomorrow_holiday_list.append(0)
            if i > 0:
                is_yesterday_holiday_list.append(0) if df.iloc[i-1]['요일'] == '월' else is_yesterday_holiday_list.append(1)
            else:
                is_yesterday_holiday_list.append(0)
        elif df.iloc[i]['요일'] == '수':
            if i < len(df)-1:
                is_tomorrow_holiday_list.append(0) if df.iloc[i+1]['요일'] == '목' else is_tomorrow_holiday_list.append(1)
            else:
                is_tomorrow_holiday_list.append(0)
            if i > 0:
                is_yesterday_holiday_list.append(0) if df.iloc[i-1]['요일'] == '화' else is_yesterday_holiday_list.append(1)
            else:
                is_yesterday_holiday_list.append(0)
        elif df.iloc[i]['요일'] == '목':
            if i < len(df)-1:
                is_tomorrow_holiday_list.append(0) if df.iloc[i+1]['요일'] == '금' else is_tomorrow_holiday_list.append(1)
            else:
                is_tomorrow_holiday_list.append(0)
            if i > 0:
                is_yesterday_holiday_list.append(0) if df.iloc[i-1]['요일'] == '수' else is_yesterday_holiday_list.append(1)
            else:
                is_yesterday_holiday_list.append(0)
        elif df.iloc[i]['요일'] == '금':
            is_tomorrow_holiday_list.append(1)
            if i > 0:
                is_yesterday_holiday_list.append(0) if df.iloc[i-1]['요일'] == '목' else is_yesterday_holiday_list.append(1)
            else:
                is_yesterday_holiday_list.append(0)
                
    return is_yesterday_holiday_list, is_tomorrow_holiday_list

In [5]:
def get_is_corona_list(df):
    corona_list = []
    corona_start = arrow.get('2020-03-01')
    for i in range(0, len(df)):
        date = arrow.get(df.iloc[i]['일자'])
        if date >= corona_start:
            corona_list.append(1)
        else:
            corona_list.append(0)
    
    return corona_list

In [6]:
def is_last_wed_of_month(date):
    curr = arrow.get(date)
    if curr.weekday() != 2:
        # not wednesday
        return False
    else:
        curr_month = curr.month
        if curr_month != curr.shift(days=7).month:
            # last wednesday of the month
            return True
        else:
            return False
        
def get_is_last_wed_list(df):
    is_last_wed_list = []
    for i in range(0, len(df)):
        date = df.iloc[i]['일자']
        if is_last_wed_of_month(date):
            is_last_wed_list.append(1)
        else:
            is_last_wed_list.append(0)
    
    return is_last_wed_list

In [7]:
def get_month_list(df):
    month_list = []
    for i in range(0, len(df)):
        month = arrow.get(df.iloc[i]['일자']).month
        month_list.append(month)
    
    return month_list

In [8]:
def get_day_list(df):
    day_list = []
    for i in range(0, len(df)):
        day = arrow.get(df.iloc[i]['일자']).day
        day_list.append(day)
    
    return day_list

In [9]:
def seperate_and_processing_menu_str(menu_row):
    # split by space
    splits = menu_row.split(' ')
    menu = []
    is_new_menu = False
    for dish in splits:
        if len(dish) > 1:
            # find (New)
            if '(New)' in dish:
                menu.append(dish.split('(New)')[0] + dish.split('(New)')[1])
                is_new_menu = True
            elif '(' not in dish and ')' not in dish:
                menu.append(dish)
    if len(menu) <= 3:
        # no menu today
        menu = ['-']
    
    return menu, is_new_menu

In [10]:
udf_delicious_lunch = [
    # meq than avg 1000, cnt 7
    '북어국',
    '돈육간장불고기',
    '모듬쌈',
    '모듬쌈*쌈장',
    '해파리겨자채',
    '옥수수계란찜',
    '쇠고기숙주볶음',
    '두부양념조림',
    '황태국',
    '유자청제육볶음',
    '소불고기',
    '홍어채무침',
    '골뱅이무침*소면',
    '제육볶음',
    '계란찜',
    '버섯불고기',
    '어묵국',
    '한방소갈비찜',
    '쫄면채소무침',
    '모둠쌈',
    '돈육김치볶음',
    '돈육간장불고기'
]

udf_delicious_dinner = [
    # meq than avg 540 cnt 7
    '수박',
    '락교',
    '우동',
    '로제스파게티',
    '오므라이스',
    '후난식볶음밥',
    '통도라지구이',
    '미니햄버거',
    '김치볶음밥',
    '가쯔오장국',
    '콘샐러드',
    '무채국',
    '해파리냉채',
    '콥샐러드',
    '취나물',
    '비트무생채'
]    

In [11]:
def get_delicious_lunch_menu(lunch_menu):
    is_delicious_lunch = []
    for menu in lunch_menu:
        is_found = False
        for dish in menu:
            if dish in udf_delicious_lunch:
                is_delicious_lunch.append(1)
                is_found = True
                break
        if not is_found:
            is_delicious_lunch.append(0)
    return is_delicious_lunch

def get_delicious_dinner_menu(dinner_menu):
    is_delicious_dinner = []
    for menu in dinner_menu:
        is_found = False
        for dish in menu:
            if dish in udf_delicious_dinner:
                is_delicious_dinner.append(1)
                is_found = True
                break
        if not is_found:
            is_delicious_dinner.append(0)
    return is_delicious_dinner

In [12]:
def get_menu_info(df):
    is_new_lunch_menu = []
    is_new_dinner_menu = []
    lunch_menus = []
    dinner_menus = []
    for i in range(0, len(df)):
        row_lunch = df.iloc[i]['중식메뉴']
        row_dinner = df.iloc[i]['석식메뉴']
        
        lunch_menu, new_lunch_menu = seperate_and_processing_menu_str(row_lunch)
        lunch_menus.append(lunch_menu)
        is_new_lunch_menu.append(1) if new_lunch_menu else is_new_lunch_menu.append(0)
        
        dinner_menu, new_dinner_menu = seperate_and_processing_menu_str(row_dinner)
        dinner_menus.append(dinner_menu)
        is_new_dinner_menu.append(1) if new_dinner_menu else is_new_dinner_menu.append(0)
        
        # to do 
        # menu classification
    is_delicious_menu_lunch_list = get_delicious_lunch_menu(lunch_menus)
    is_delicious_menu_dinner_list = get_delicious_dinner_menu(dinner_menus)
    
    return is_new_lunch_menu, is_new_dinner_menu, is_delicious_menu_lunch_list, is_delicious_menu_dinner_list

In [13]:
def get_is_no_dinner_list(df):
    is_no_dinner_list = []
    for i in range(0, len(df)):
        is_no_dinner_list.append(1) if len(df.iloc[i]['석식메뉴']) <= 20 else is_no_dinner_list.append(0)
    
    return is_no_dinner_list

In [14]:
def pre_processing(df):
    yesterday_holiday_list, tommorow_holiday_list = get_is_holiday_lists(df)
    is_corona_list = get_is_corona_list(df)
    month_list = get_month_list(df)
    day_list = get_day_list(df)
    is_no_dinner_list = get_is_no_dinner_list(df)
    is_new_lunch_list, is_new_dinner_list, is_delicious_menu_lunch_list, is_delicious_menu_dinner_list = get_menu_info(df)
    is_last_wed_list = get_is_last_wed_list(df)

    is_yesterday_holiday = pd.DataFrame({'is_yesterday_holiday': yesterday_holiday_list})
    is_tomorrow_holiday = pd.DataFrame({'is_tomorrow_holiday': tommorow_holiday_list})
    is_corona = pd.DataFrame({'is_corona': is_corona_list})
    month = pd.DataFrame({'month': month_list})
    day = pd.DataFrame({'day': day_list})
    no_dinner = pd.DataFrame({'no_dinner': is_no_dinner_list})
    is_new_lunch = pd.DataFrame({'is_new_lunch': is_new_lunch_list})
    is_new_dinner = pd.DataFrame({'is_new_dinner': is_new_dinner_list})
    is_delicious_lunch = pd.DataFrame({'is_delicious_lunch': is_delicious_menu_lunch_list})
    is_delicious_dinner = pd.DataFrame({'is_delicious_dinner': is_delicious_menu_dinner_list})
    #is_last_wed = pd.DataFrame({'is_last_wed': is_last_wed_list})

    df = df.join(is_yesterday_holiday)
    df = df.join(is_tomorrow_holiday)
    df = df.join(is_corona)
    df = df.join(month)
    df = df.join(day)
    df = df.join(no_dinner)
    #df = df.join(is_new_lunch)
    df = df.join(is_new_dinner)
    #df = df.join(is_delicious_lunch)
    df = df.join(is_delicious_dinner)
    #df = df.join(is_last_wed)

    df = df.drop(['일자', '조식메뉴', '중식메뉴', '석식메뉴'],axis='columns')

    # 원핫인코딩
    df = pd.get_dummies(df, columns=['요일'])
    
    return df

In [15]:
df_train = pre_processing(df_train)

In [16]:
df_train.head()

,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,precipitation,avg_temperature,min_temperature,...,day,no_dinner,is_new_dinner,is_delicious_lunch,is_delicious_dinner,요일_금,요일_목,요일_수,요일_월,요일_화
0,2601,50,150,238,0.0,1039.0,331.0,0.0,-0.6,-4.1,...,1,0,0,1,0,0,0,0,1,0
1,2601,50,173,319,0.0,867.0,560.0,0.0,-2.3,-9.1,...,2,0,0,0,0,0,0,0,0,1
2,2601,56,180,111,0.0,1017.0,573.0,0.0,-1.7,-9.3,...,3,0,0,0,0,0,0,1,0,0
3,2601,104,220,355,0.0,978.0,525.0,0.0,-0.2,-7.7,...,4,0,0,0,1,0,1,0,0,0
4,2601,278,181,34,0.0,925.0,330.0,0.0,1.3,-6.5,...,5,0,0,0,0,1,0,0,0,0


In [17]:
from sklearn.model_selection import train_test_split
import random

x1_train, x1_test, y1_train, y1_test = train_test_split(df_train.loc[:, [col for col in df_train.columns if col != '중식계' and col != '석식계']], df_train['중식계'],
                                                        random_state=random.randrange(1, 100))
x2_train, x2_test, y2_train, y2_test = train_test_split(df_train.loc[:, [col for col in df_train.columns if col != '중식계' and col != '석식계']], df_train['석식계'],
                                                        random_state=random.randrange(1, 100))

In [18]:

# AutoML을 이용한 ML 구현
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold, GridSearchCV
# suboptmal problem의 iteration 문제는 무시할 것! (suboptimal의 계산횟수를 한정하여 수렴시키겠다는 얘기)
# 궁금하면 max_iter라는 값을 인자로 전달받는 ML 알고리즘을 살펴보세요!
import warnings
warnings.filterwarnings('ignore')

# 필요한 알고리즘을 불러와야 합니다!
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
# Ridge와 Lasso의 hyperparmeter: alpha
# Logistic Regressio의 hyperparameter: penalty와 regularization strength인 C값
from sklearn.svm import SVR
# SVR의 hyperparameter: epsilon, regularization C, gamma, kernel = 'rbf', 'poly', 'sigmoid'
from sklearn.neural_network import MLPRegressor
# hidden_layer_sizes = (100,) , (10, 10, ) 정도만 activation = 'relu', 'logistic'까지만, alpha =0.0001, solver = 'lbfgs', 'adam'까지만

In [19]:
pipe = Pipeline([('preprocessing', None), ('regressor', LinearRegression())])
pre_list = [StandardScaler(), MinMaxScaler(), None]
hyperparam_grid = [
    {"regressor": [LinearRegression()], 'preprocessing': pre_list},
    {"regressor": [Ridge()], 'preprocessing': pre_list,
    "regressor__alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
    {"regressor": [Lasso()], 'preprocessing': pre_list,
    "regressor__alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
    {"regressor": [LogisticRegression()], 'preprocessing': pre_list,
    "regressor__C": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
    {"regressor": [SVR()], 'preprocessing': pre_list,
    "regressor__epsilon": [0.001, 0.01, 0.1, 1, 10],
    "regressor__C": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
    {"regressor": [MLPRegressor()], 'preprocessing': pre_list,
    "regressor__hidden_layer_sizes": [(100,) , (10, 10, )],
    "regressor__activation": ["relu", "logistic"],
    "regressor__solver": ["lbfgs", "adam"],
    "regressor__alpha": [0.0001, 0.01, 1]}

]
kfold = KFold(n_splits=5, shuffle=True, random_state=1)

grid1 = GridSearchCV(pipe, hyperparam_grid, scoring="neg_mean_absolute_error",
                   refit=True, cv=kfold)
grid2 = GridSearchCV(pipe, hyperparam_grid, scoring="neg_mean_absolute_error",
                   refit=True, cv=kfold)

In [20]:
grid1.fit(x1_train, y1_train)
print(grid1.best_estimator_)
print(grid1.best_params_)
print(-grid1.best_score_)
print(-grid1.score(x1_test, y1_test))

Pipeline(steps=[('preprocessing', MinMaxScaler()),
                ('regressor',
                 MLPRegressor(alpha=0.01, hidden_layer_sizes=(10, 10),
                              solver='lbfgs'))])
{'preprocessing': MinMaxScaler(), 'regressor': MLPRegressor(alpha=0.01, hidden_layer_sizes=(10, 10), solver='lbfgs'), 'regressor__activation': 'relu', 'regressor__alpha': 0.01, 'regressor__hidden_layer_sizes': (10, 10), 'regressor__solver': 'lbfgs'}
72.224603350013
67.88306377010703


In [21]:
grid2.fit(x2_train, y2_train)
print(grid2.best_estimator_)
print(grid2.best_params_)
print(-grid2.best_score_)
print(-grid2.score(x2_test, y2_test))

Pipeline(steps=[('preprocessing', StandardScaler()),
                ('regressor', Lasso(alpha=1))])
{'preprocessing': StandardScaler(), 'regressor': Lasso(alpha=1), 'regressor__alpha': 1}
52.78012302287361
51.69728037223961


In [22]:
from sklearn.metrics import mean_absolute_error
print("중식 최종 결과: ", mean_absolute_error(y1_test, grid1.best_estimator_.predict(x1_test)))
print("석식 최종 결과: ", mean_absolute_error(y2_test, grid2.best_estimator_.predict(x2_test)))

중식 최종 결과:  67.88306377010703
석식 최종 결과:  51.69728037223961


In [23]:
df_test = pre_processing(df_test)

lunch_cnt_list = grid1.best_estimator_.predict(df_test)
dinner_cnt_list = grid2.best_estimator_.predict(df_test)

In [24]:
df_result_lunch = pd.DataFrame({'중식계': lunch_cnt_list})
df_result_dinner = pd.DataFrame({'석식계': dinner_cnt_list})

In [25]:
df_result = df_result_date.join(df_result_lunch).join(df_result_dinner)

In [26]:
df_result.to_csv('../../dataset/meal_service/result.csv', index=None)

In [27]:
df_result

,일자,중식계,석식계
0,2021-01-27,936.715055,374.721746
1,2021-01-28,854.925445,428.138567
2,2021-01-29,657.787178,300.858884
3,2021-02-01,1240.633903,526.349721
4,2021-02-02,1030.707798,468.726666
5,2021-02-03,1023.313391,391.956466
6,2021-02-04,953.598668,461.857744
7,2021-02-05,726.809573,338.877025
8,2021-02-08,1248.992977,546.635916
9,2021-02-09,1012.417098,486.771824


In [28]:
print('job running time %f sec' % (time.time() - job_start_time ))

job running time 330.201537 sec
